In [1]:
import numpy as np

import cirq
from cirq import ops
from cirq.ops import CZ, H, CNOT, X, Y, Z, RotYGate, MeasurementGate
from cirq.google import XmonQubit, ExpWGate, ExpZGate, Exp11Gate, XmonMeasurementGate
from cirq.google import Simulator, ConvertToXmonGates, MergeRotations, MergeInteractions

In [116]:
class Basis:
    gate_list = (ops.X, ops.Y, ops.Z)
    def __init__(self, axis: int):
        assert 0 <= axis < 6, 'Axis out of bounds.'
        self.axis = axis
    def to_basis_ops(self, new_basis: 'Basis', qubit: cirq.QubitId):
        return (gate.on(qubit) for gate in self.to_basis_gates(new_basis))
    def to_basis_gates(self, new_basis: 'Basis'):
        #return (self.to_x, self.to_y, self.to_z)[new_basis.axis]()
        axis = self.axis
        axis2 = new_basis.axis
        #if axis == axis2:
        #    return ()
        #assert axis2 < 3 and axis < 3, 'Temp'
        flip = (axis < 3) != (axis2 < 3)
        #axis = axis % 3
        #axis2 = axis2 % 3
        cycle = (axis + axis2) % 3
        gate_i = (-cycle) % 3
        quarter_turns = (axis2 - axis + 1) % 3 - 1  # -1, 0, or +1
        if flip:
            quarter_turns = -quarter_turns
        if quarter_turns == 0:
            if flip:
                gate = self.gate_list[(axis + 1) % 3]  # Or +2
                return (gate,)
            else:
                return ()
        else:
            gate = self.gate_list[gate_i] ** (quarter_turns * 0.5)
            return (gate,)
    def to_x(self):
        if self.axis == 0:
            return ()
        elif self.axis == 1:
            return (ops.Z ** -0.5,)
        elif self.axis == 2:
            return (ops.Y ** 0.5,)
    def to_y(self):
        if self.axis == 0:
            return (ops.Z ** 0.5,)
        elif self.axis == 1:
            return ()
        elif self.axis == 2:
            return (ops.X ** -0.5,)
    def to_z(self):
        if self.axis == 0:
            return (ops.Y ** -0.5,)
        elif self.axis == 1:
            return (ops.X ** 0.5,)
        elif self.axis == 2:
            return ()

    def pauli_gate(cls, axis: int):
        return cls.gate_list[axis]

X_BASIS = Basis(0)
Y_BASIS = Basis(1)
Z_BASIS = Basis(2)
nX_BASIS = Basis(3)
nY_BASIS = Basis(4)
nZ_BASIS = Basis(5)

In [93]:
class InteractionGate(cirq.CompositeGate, cirq.TextDiagrammableGate):
    def __init__(self, basis0: Basis, basis1: Basis):
        self.basis0 = basis0
        self.basis1 = basis1

    def default_decompose(self, qubits):
        q0, q1 = qubits
        yield from self.basis0.to_basis_ops(Z_BASIS, q0)
        yield from self.basis1.to_basis_ops(Z_BASIS, q1)
        yield ops.CZ(q0, q1)
        yield from Z_BASIS.to_basis_ops(self.basis0, q0)
        yield from Z_BASIS.to_basis_ops(self.basis1, q1)

    def text_diagram_wire_symbols(self, qubit_count=None, use_unicode_characters=True, precision=3):
        labels = ('X', 'Y', 'Z', 'nX', 'nY', 'nZ')
        return (labels[self.basis0.axis], labels[self.basis1.axis])

In [94]:
for b0 in range(6):
    for b1 in range(6):
        gates = Basis(b0).to_basis_gates(Basis(b1))
        print(gates)

()
(Z**0.5,)
(Y**-0.5,)
(Y,)
(Z**-0.5,)
(Y**0.5,)
(Z**-0.5,)
()
(X**0.5,)
(Z**0.5,)
(Z,)
(X**-0.5,)
(Y**0.5,)
(X**-0.5,)
()
(Y**-0.5,)
(X**0.5,)
(X,)
(Y,)
(Z**-0.5,)
(Y**0.5,)
()
(Z**0.5,)
(Y**-0.5,)
(Z**0.5,)
(Z,)
(X**-0.5,)
(Z**-0.5,)
()
(X**0.5,)
(Y**-0.5,)
(X**0.5,)
(X,)
(Y**0.5,)
(X**-0.5,)
()


In [95]:
q0, q1, q2 = (cirq.NamedQubit('q{}'.format(i)) for i in range(3))
def opsToMatrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    m = c.to_unitary_matrix()
    print(c)
    return m.round(5)

In [115]:
opsToMatrix(Z(q1), InteractionGate(Z_BASIS, X_BASIS)(q0, q1), Z(q1))

q0: ───────Z───────
           │
q1: ───Z───X───Z───


array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j]])

In [107]:
nZ_BASIS.to_basis_gates(Z_BASIS)

(X,)

In [126]:
opsToMatrix(
    InteractionGate(nX_BASIS, Z_BASIS)(q0, q1),
    InteractionGate(Z_BASIS, nX_BASIS)(q0, q1),
    InteractionGate(nX_BASIS, Z_BASIS)(q0, q1),
)

q0: ───nZ───Z────nZ───
       │    │    │
q1: ───Z────nZ───Z────


array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]])

In [26]:
sq2 = np.sqrt(2)
mX = X.matrix()
mY = Y.matrix()
mZ = Z.matrix()
mH = H.matrix()

In [27]:
mX_2 = (X**0.5).matrix()

In [28]:
mY_2 = (Y**0.5).matrix()

In [29]:
mZ_2 = (Z**0.5).matrix().round(3)

In [30]:
#mZX = opsToMatrix(CNOT(q0, q1))
mZX = opsToMatrix(ZX()(q0, q1))
mZX

q0: ───<__main__.ZX object at 0x7fb46e830b38>:0───
       │
q1: ───<__main__.ZX object at 0x7fb46e830b38>:1───


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]])

In [31]:
mZZ = opsToMatrix(Y(q1)**0.5, CNOT(q0, q1), Y(q1)**-0.5)
mZZ

q0: ───────────@────────────
               │
q1: ───Y^0.5───X───Y^-0.5───


array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [32]:
mXZ = opsToMatrix(Y(q0)**-0.5, CZ(q0, q1), Y(q0)**0.5)
mXZ

q0: ───Y^-0.5───@───Y^0.5───
                │
q1: ────────────Z───────────


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]])

In [38]:
mH = opsToMatrix(H(q0))      #/(1/sq2-1j/sq2)
mH

q0: ───H───


array([[ 0.70711+0.j,  0.70711+0.j],
       [ 0.70711+0.j, -0.70711+0.j]])

In [39]:
mH = opsToMatrix(Y(q0)**-0.5, Z(q0))      /(1/sq2-1j/sq2)
mH

q0: ───Y^-0.5───Z───


array([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]])

In [35]:
mH = opsToMatrix(Z(q0), Y(q0)**0.5)      /(1/sq2+1j/sq2)
mH

q0: ───Z───Y^0.5───


array([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]])

In [36]:
mH = opsToMatrix(Y(q0)**0.5, X(q0))      /(1/sq2+1j/sq2)
mH

q0: ───Y^0.5───X───


array([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]])

In [37]:
mH = opsToMatrix(X(q0), Y(q0)**-0.5)      /(1/sq2-1j/sq2)
mH

q0: ───X───Y^-0.5───


array([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]])

In [27]:
mH.dot(mX)*sq2

array([[ 1.+0.j,  1.+0.j],
       [-1.+0.j,  1.+0.j]])

In [13]:
mZ.dot(mH)*sq2

NameError: name 'mZ' is not defined

In [29]:
mH.dot(mX)*sq2

array([[ 1.+0.j,  1.+0.j],
       [-1.+0.j,  1.+0.j]])

In [32]:
mZ.dot(mH) .dot( mX.dot(mH)) .round(3)

array([[ 1.+0.j, -0.+0.j],
       [-0.+0.j,  1.+0.j]])

In [33]:
mY_2.dot(mX)

array([[-0.5-0.5j,  0.5+0.5j],
       [ 0.5+0.5j,  0.5+0.5j]])

In [35]:
mZ.dot(mY_2) * -1

array([[-0.5-0.5j,  0.5+0.5j],
       [ 0.5+0.5j,  0.5+0.5j]])